Import lib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.utils import to_categorical
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

Load dataset

In [ ]:
train_df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/per_api_drebin_amd_benign_filter40/train-0.csv', header=None, skiprows=1)
val_df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/per_api_drebin_amd_benign_filter40/file-0.csv', header=None, skiprows=1)
test_df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/per_api_drebin_amd_benign_filter40/file-1.csv', header=None, skiprows=1)

train_x = np.array(train_df.iloc[:, 3:])
train_y = np.array(train_df.iloc[:, 2])

val_x = np.array(val_df.iloc[:, 3:])
val_y = np.array(val_df.iloc[:, 2])

test_x = np.array(test_df.iloc[:, 3:])
test_y = np.array(test_df.iloc[:, 2])

1CNN

In [ ]:
def cnn_model(X_train, y_train, X_val, y_val):
  '''Complie, train and get report of model CNN.
  Input:
    X_train: Feauters of train dataset
    y_train: Labels of train dataset
    X_val: Feauters of validate dataset
    y_train: Labels of validate dataset
  Ouput:
    model: Model 1CNN
  '''
  BATCH_SIZE = 512
  FEUTURES_SIZE = 44
  PADDING_SIZE = 28
  N_CLASSES = 228
  LR = 0.001
  N_EPOCHS = 18

  #padding
  train_x = np.concatenate((X_train, np.zeros((X_train.shape[0], PADDING_SIZE))),1)
  val_x = np.concatenate((X_val, np.zeros((X_val.shape[0], PADDING_SIZE))),1)

  #transform data
  train_x_cnn = train_x.reshape(train_x.shape[0], FEUTURES_SIZE, FEUTURES_SIZE, 1)
  val_x_cnn = val_x.reshape(val_x.shape[0], FEUTURES_SIZE, FEUTURES_SIZE, 1)

  train_y_cnn = to_categorical(y_train, N_CLASSES)
  val_y_cnn = to_categorical(y_val, N_CLASSES)

  #complie model
  input_cnn = Input(shape=(FEUTURES_SIZE, FEUTURES_SIZE,1))
  conv1 = Conv2D(32, kernel_size=2, activation='relu', padding="same", input_shape=(FEUTURES_SIZE, FEUTURES_SIZE, 1))(input_cnn)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

  conv2 = Conv2D(32, kernel_size=2, activation='relu', padding="same")(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

  conv3 = Conv2D(64, kernel_size=2, activation='relu', padding="same")(pool2)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

  dropout5 = Dropout(0.15)(pool3)
  flatten_cnn = Flatten()(dropout5)

  hidden1 = Dense(2048, activation='relu')(flatten_cnn)
  hidden2 = Dense(1024, activation='relu')(hidden1)

  dropout3 = Dropout(0.7)(hidden2)
  output_cnn = Dense(N_CLASSES, activation='softmax')(dropout3)

  model_cnn = Model(inputs=input_cnn, outputs=output_cnn, name="model_cnn")
  model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  #train
  tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
  model_cnn.fit(train_x_cnn, train_y_cnn, epochs=N_EPOCHS, batch_size=BATCH_SIZE ,
                      validation_data=(val_x_cnn, val_y_cnn), callbacks=[tensorBoardCallback])
  
  test_logits = model_cnn.predict(val_x_cnn)
  #Lấy phần tử có giá trị lớn nhất
  y_pred = np.argmax(test_logits, axis=-1)
  #Kết quả thu được
  rp = classification_report(y_val, y_pred, output_dict=True)
  del rp['accuracy'], rp['macro avg'], rp['weighted avg']
  weights = [0] * 228
  for key in rp:
    weights[int(key)] = float(rp[key]['precision']) 
  print(weights)
  
  return model_cnn

In [ ]:
cnn = cnn_model(train_x, train_y, val_x, val_y)

Epoch 1/18
 1/49 [..............................] - ETA: 0s - loss: 5.4426 - accuracy: 0.0020WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
49/49 [==============================] - 1s 20ms/step - loss: 2.2995 - accuracy: 0.4454 - val_loss: 1.0962 - val_accuracy: 0.7165
Epoch 2/18
49/49 [==============================] - 1s 13ms/step - loss: 0.8492 - accuracy: 0.7734 - val_loss: 0.5063 - val_accuracy: 0.8491
Epoch 3/18
49/49 [==============================] - 1s 14ms/step - loss: 0.4920 - accuracy: 0.8600 - val_loss: 0.3238 - val_accuracy: 0.9009
Epoch 4/18
49/49 [==============================] - 1s 14ms/step - loss: 0.3485 - accuracy: 0.8968 - val_loss: 0.2587 - val_accuracy: 0.9179
Epoch 5/18
49/49 [==============================] - 1s 13ms/step - loss: 0.2

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rfc10 = RandomForestClassifier(max_depth=10)
history_rfc10 = model_rfc10.fit(train_x, train_y)

model_rfc50 = RandomForestClassifier(max_depth=50)
history_rfc50 = model_rfc50.fit(train_x, train_y)

model_rfc100 = RandomForestClassifier(max_depth=20)
history_rfc100 = model_rfc100.fit(train_x, train_y)

y_pred = history_rfc10.predict(val_x)
rp = classification_report(val_y, y_pred, output_dict=True)
del rp['accuracy'], rp['macro avg'], rp['weighted avg']
weights = [0] * 228
for key in rp:
  weights[int(key)] = float(rp[key]['precision'])
print(weights)
  


y_pred = history_rfc50.predict(val_x)
rp = classification_report(val_y, y_pred, output_dict=True)
del rp['accuracy'], rp['macro avg'], rp['weighted avg']
weights = [0] * 228
for key in rp:
  weights[int(key)] = float(rp[key]['precision'])
print(weights)



y_pred = history_rfc100.predict(val_x)
rp = classification_report(val_y, y_pred, output_dict=True)
del rp['accuracy'], rp['macro avg'], rp['weighted avg']
weights = [0] * 228
for key in rp:
  weights[int(key)] = float(rp[key]['precision'])
print(weights)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.9768451519536903, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.7142857142857143, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0.9831932773109243, 0, 0, 0, 0, 0, 0.0, 0, 1.0, 0, 0.9666666666666667, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0.8, 0, 0.0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.6666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9567307692307693, 1.0, 1.0, 0, 1.0, 1.0, 0, 0.8067632850241546, 1.0, 0, 0, 0.7678571428571429, 0, 1.0, 1.0, 0.0, 1.0, 0.9464285714285714, 1.0, 0.9140625, 1.0, 0, 0.8260869565217391, 1.0, 1.0, 0, 1.0, 1.0, 0.0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.9253731343283582, 0, 0]
[0.9897058823529412, 0, 0, 0, 0.75, 0, 0, 0,

In [ ]:
RFC_WEIGHTS10 = [0.974025974025974, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.6666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0.9915254237288136, 0, 0, 0, 0, 0, 0.0, 0, 1.0, 0, 0.9666666666666667, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0.7777777777777778, 0, 0.0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.9838709677419355, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9543269230769231, 1.0, 1.0, 0, 1.0, 1.0, 0, 0.8141809290953546, 1.0, 0, 0, 0.7706093189964157, 0, 1.0, 1.0, 0.0, 1.0, 0.9122807017543859, 1.0, 0.928, 1.0, 0, 0.8260869565217391, 1.0, 1.0, 0, 1.0, 1.0, 1.0, 0, 0, 0, 1.0, 1.0, 0.9444444444444444, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.9057971014492754, 0, 0]
RFC_WEIGHTS50 = [0.9897360703812317, 0, 0, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.6666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0.9915254237288136, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 1.0, 0, 0.6842105263157895, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0.7941176470588235, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9778325123152709, 1.0, 1.0, 0, 1.0, 1.0, 0, 0.8723958333333334, 1.0, 0, 0, 0.8318965517241379, 0, 1.0, 0.6666666666666666, 0.0, 1.0, 0.9473684210526315, 1.0, 0.975, 1.0, 0, 0.8214285714285714, 1.0, 1.0, 0, 1.0, 1.0, 1.0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.9618320610687023, 0, 0]
RFC_WEIGHTS100 = [0.9897360703812317, 0, 0, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.6666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0.9915254237288136, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 1.0, 0, 0.6842105263157895, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0.7941176470588235, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9778325123152709, 1.0, 1.0, 0, 1.0, 1.0, 0, 0.8723958333333334, 1.0, 0, 0, 0.8318965517241379, 0, 1.0, 0.6666666666666666, 0.0, 1.0, 0.9473684210526315, 1.0, 0.975, 1.0, 0, 0.8214285714285714, 1.0, 1.0, 0, 1.0, 1.0, 1.0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.9618320610687023, 0, 0]
CNN_WEIGHTS = [0.997037037037037, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0.9375, 0, 0, 0, 0, 0, 0.7142857142857143, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0.9669421487603306, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 1.0, 0, 0.6883116883116883, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0.875, 0, 0.5, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0.75, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.9838709677419355, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9824561403508771, 1.0, 1.0, 0, 1.0, 0.95, 0, 0.919889502762431, 1.0, 0, 0, 0.8318965517241379, 0, 1.0, 0.7777777777777778, 0.0, 0.9333333333333333, 1.0, 1.0, 0.9658119658119658, 1.0, 0, 0.8846153846153846, 1.0, 0.9047619047619048, 0, 1.0, 1.0, 1.0, 0, 0, 0, 1.0, 0.85, 1.0, 0.9333333333333333, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0.975, 0, 0]

In [ ]:
class CustomSoftVotingClassifier:
  def __init__(self, models=[], weights=[]):
    self.models = models
    self.weights = weights

  def __check(self):
    if (len(self.models) != len(self.weights)):
      return False
    return True

  def predict(self, X_test):
    if self.__check():
      output = []
      lbs1 = history_rfc10.predict(X_test)
      print("RFC10: ", accuracy_score(lbs1, test_y))
      lbs2 = history_rfc50.predict(X_test)
      print("RFC50: ", accuracy_score(lbs2, test_y))

      lbs3 = history_rfc100.predict(X_test)
      print("RFC100: ", accuracy_score(lbs3, test_y))
      
      # X_test = np.concatenate((X_test, np.zeros((X_test.shape[0], 28))),1)
      # X_test = X_test.reshape(X_test.shape[0], 44, 44, 1)
      # lbs3 = np.argmax(cnn.predict(X_test), axis=-1)
      # print("CNN: ", accuracy_score(lbs3, test_y))
      
      tmp1 = [self.weights[1][lb] for lb in lbs1]
      tmp2 = [self.weights[2][lb] for lb in lbs2]
      tmp3 = [self.weights[0][lb] for lb in lbs3]
      
      for i in range(len(tmp1)):
        if tmp1[i] == tmp2[i] == tmp3[i]:
          tmp = [lbs1[i], lbs2[i], lbs3[i]]
          if tmp.count(lbs1[i]) >= 2:
            output.append(lbs1[i])
          elif tmp.count(lbs2[i]) >= 2:
            output.append(lbs2[i])
          else:
            output.append(lbs3[i])
        else:
          m = max(tmp1[i], tmp2[i], tmp3[i])
          if m == tmp3[i]:
            output.append(lbs3[i])
          elif m == tmp1[i]:
            output.append(lbs1[i])
          else:
            output.append(lbs2[i])

      return output
    else:
      print("len(models) != len(weights)")


In [ ]:
# csvc = CustomSoftVotingClassifier([cnn, history_rfc10, history_rfc50], [CNN_WEIGHTS, RFC_WEIGHTS10, RFC_WEIGHTS50])
csvc = CustomSoftVotingClassifier([history_rfc100, history_rfc10, history_rfc50], [RFC_WEIGHTS100, RFC_WEIGHTS10, RFC_WEIGHTS50])
y_pred = csvc.predict(test_x)

RFC10:  0.9244053437601825
RFC50:  0.946236559139785
RFC100:  0.9475399152818508


In [ ]:
report = classification_report(test_y, y_pred)
print(report)
print(accuracy_score(y_pred, test_y))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       677
           4       0.82      1.00      0.90         9
          12       1.00      0.91      0.95        32
          18       0.67      0.57      0.62         7
          32       1.00      1.00      1.00         8
          33       0.99      0.97      0.98       121
          39       0.50      0.50      0.50         6
          41       1.00      1.00      1.00        15
          43       0.69      0.61      0.65        92
          48       1.00      1.00      1.00         6
          64       1.00      1.00      1.00         5
          69       0.77      0.82      0.79        33
          71       0.17      0.17      0.17         6
          77       1.00      1.00      1.00        15
          78       1.00      1.00      1.00         4
          85       1.00      1.00      1.00        14
          87       1.00      0.50      0.67         4
         104       1.00    

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
eclf3 = VotingClassifier(estimators=[('lr', model_rfc10), ('rf', model_rfc50), ('gnb', model_rfc100)], voting='hard', weights=[1,1,1], flatten_transform=True)
eclf3 = eclf3.fit(train_x, train_y)
y_pred = eclf3.predict(test_x)
print(accuracy_score(y_pred, test_y))

0.9472140762463344


In [ ]:
print(y_pred)
print(list(test_y))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 187, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 180, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 180, 181, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 180, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,